In [ ]:
!pip install transformers
!pip install torch torchvision
!pip install pillow
!pip install tqdm


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c 'petfinder-adoption-prediction'

Saving kaggle.json to kaggle.json
rm: cannot remove '/root/.kaggle': No such file or directory
 99% 1.92G/1.94G [00:24<00:00, 102MB/s] 
100% 1.94G/1.94G [00:24<00:00, 86.0MB/s]


In [ ]:
# Unzip the downloaded data
import zipfile
data_zip_path = "/content/petfinder-adoption-prediction.zip"
with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
  zip_ref.extractall("/content")  # Extract to the same directory

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
# Ensure the processed data directory exists in Google Drive
processed_data_dir = '/content/drive/MyDrive/ProcessedData'
if not os.path.exists(processed_data_dir):
    os.makedirs(processed_data_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
image_folder = "/content/train_images"

In [ ]:
image_folder_test = "/content/test_images"

In [ ]:
from PIL import Image
import torch

from transformers import CLIPProcessor, CLIPModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [ ]:
from tqdm import tqdm
import os

def extract_image_quality(image_folder, model, processor, device):

  labels = [
      "High quality picture of a pet",
      "Well-focused picture of a pet",
  ]
  images = os.listdir(image_folder)
  images.sort()

  # group images by PetId
  image_groups = {}
  for image_name in images:
      pet_id = image_name.split('-')[0]
      if pet_id not in image_groups:
          image_groups[pet_id] = []

      image_groups[pet_id].append(image_name)


  pet_id_features = {}

  items = image_groups.items()
  for (pet_id, images) in tqdm(items):


      # open images and create a list of images for this pet_id
      image_paths = [os.path.join(image_folder, image_name) for image_name in images]
      images = [Image.open(image_path).convert('RGB') for image_path in image_paths]
      inputs = processor(text=labels, images=images, return_tensors="pt", padding=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}

      outputs = model(**inputs)

      mean_scores = outputs.logits_per_image.mean(dim=0)
      probs = torch.softmax(mean_scores, dim=0)
      probs = probs.cpu()
      values = list(probs.detach().numpy())
      values_dict = dict(zip(labels, values))
      pet_id_features[pet_id] = values_dict


  return pet_id_features


In [ ]:
pet_images_quality = extract_image_quality(image_folder, model, processor, device)

100%|██████████| 14652/14652 [15:18<00:00, 15.95it/s]


In [ ]:
print(pet_images_quality['0008c5398'])

{'High quality picture of a pet': 0.51731086, 'Well-focused picture of a pet': 0.48268914}


In [ ]:
import pandas as pd
pet_image_quality_df = pd.DataFrame.from_dict(pet_images_quality, orient='index')
pet_image_quality_df = pet_image_quality_df.reset_index()
pet_image_quality_df.columns = ['PetID', 'HighQualityImage', 'WellFocusedImage']
pet_image_quality_df.head()

,PetID,HighQualityImage,WellFocusedImage
0,0008c5398,0.517311,0.482689
1,000a290e4,0.507919,0.492081
2,000fb9572,0.144271,0.855729
3,0011d7c25,0.072677,0.927323
4,00156db4a,0.138572,0.861428


In [ ]:
checkpoint_path_quality = '/content/drive/MyDrive/ProcessedData/pet_images_quality.csv'
pet_image_quality_df.to_csv(checkpoint_path_quality, index=False)


In [ ]:
pet_images_quality_test = extract_image_quality(image_folder_test, model, processor, device)
import pandas as pd
pet_images_quality_test_df = pd.DataFrame.from_dict(pet_images_quality_test, orient='index')
pet_images_quality_test_df = pet_images_quality_test_df.reset_index()
pet_images_quality_test_df.columns = ['PetID', 'HighQualityImage', 'WellFocusedImage']
pet_images_quality_test_df.head()

In [ ]:
checkpoint_path_quality_test = '/content/drive/MyDrive/ProcessedData/pet_images_quality_test.csv'
pet_images_quality_test_df.to_csv(checkpoint_path_quality_test, index=False)